In [9]:
import os, sys, subprocess

`base_dir`: where the logs are stored.<br>
`identifier`: all dirs with this in their names will be selected.<br>
`excluders`: all dirs with these in their names will be excluded.<br>
`job_num`: check one job num or all. (`*`).<br>
`err`: error to select files based on.

In [10]:
base_dir = '/work/submit/lavezzo/SUEP/logs/'
identifiers = ['inf']
excluders = ['inf_data', 'inf_data_v2','inf_data_v3', 'QCD']
err = '"A process in the process pool was terminated abruptly"'
job_num = '*'      # set to * for any

In [11]:
dirs = [name for name in os.listdir(base_dir) if all([i in name for i in identifiers])]
dirs = [name for name in dirs if not any([excl in name for excl in excluders])]
# these are the directories selected to be analyzed
dirs

['jobs_inf_SUEP-m750-darkPho+RunIIAutumn18-private+MINIAODSIM',
 'jobs_inf_EGamma+Run2018C-UL2018_MiniAODv2-v1+MINIAOD',
 'jobs_inf_SUEP-m1000-darkPho+RunIIAutumn18-private+MINIAODSIM',
 'jobs_inf_SUEP-m125-generic+RunIIAutumn18-private+MINIAODSIM',
 'jobs_inf_SUEP-m1000-darkPhoHad+RunIIAutumn18-private+MINIAODSIM',
 'jobs_inf_JetHT+Run2018A-UL2018_MiniAODv2-v1+MINIAOD',
 'jobs_inf_SingleMuon+Run2018C-UL2018_MiniAODv2-v2+MINIAOD',
 'jobs_inf_JetHT+Run2018B-UL2018_MiniAODv2-v1+MINIAOD',
 'jobs_inf_SUEP-m400-generic+RunIIAutumn18-private+MINIAODSIM',
 'jobs_inf_SingleMuon+Run2018A-UL2018_MiniAODv2-v3+MINIAOD',
 'jobs_inf_EGamma+Run2018B-UL2018_MiniAODv2-v1+MINIAOD',
 'jobs_inf_JetHT+Run2018C-UL2018_MiniAODv2-v1+MINIAOD',
 'jobs_inf_SingleMuon+Run2018B-UL2018_MiniAODv2-v2+MINIAOD',
 'jobs_inf_SUEP-m400-darkPhoHad+RunIIAutumn18-private+MINIAODSIM',
 'jobs_inf_SUEP-m1000-generic+RunIIAutumn18-private+MINIAODSIM',
 'jobs_inf_SUEP-m750-darkPhoHad+RunIIAutumn18-private+MINIAODSIM',
 'jobs_inf_

In [12]:
# How many files have this error
for i,dir_name in enumerate(dirs):
    print(dir_name)
    error_files = os.popen('grep '+ err + ' ' + base_dir + dir_name +'/' + job_num + '.*.err').readlines()
    print(len(error_files))

jobs_inf_SUEP-m750-darkPho+RunIIAutumn18-private+MINIAODSIM
40
jobs_inf_EGamma+Run2018C-UL2018_MiniAODv2-v1+MINIAOD
392
jobs_inf_SUEP-m1000-darkPho+RunIIAutumn18-private+MINIAODSIM


KeyboardInterrupt: 

`WARNING`: this cell will **delete** files.<br>
This will find all files with the specific error in the .err, and delete the correspdoning HDF5 file.<br>
This might need to be adjusted based on the error / log files outputs. First run it dry to make sure it works.

In [13]:
base_out_dir = '/mnt/T3_US_MIT/hadoop/scratch/lavezzo/SUEP/inf/'
dry = False

for i, dir_name in enumerate(dirs):
    print(dir_name)
    # find .err files with error
    error_files = os.popen('grep '+ err + ' ' + base_dir + dir_name +'/' + job_num + '.*.err').readlines()
    print(len(error_files))
    for f in error_files:
        
        # find corresponding .out files
        outfile = f.split(":")[0][:-3]+"out"
        
        # grab name of .root file
        file_hdf = os.popen('grep "xrdcp root" ' + outfile).readlines()[0].split("/")[-1].split(" ")[0]
        file_hdf = file_hdf.split(".root")[0]+".hdf5"
        
        # run it dry to make sure you are deleting what you want to delete
        if dry:
            print(['rm',base_out_dir+dir_name.split(identifier)[-1][1:]+"/"+file_hdf])
            break
            
        # delete the hdf5 file
        else:
            try: 
                _ = subprocess.run(['rm',base_out_dir+dir_name.split(identifier)[-1][1:]+"/"+file_hdf],
                              stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
            except:
                continue

jobs_inf_SUEP-m750-darkPho+RunIIAutumn18-private+MINIAODSIM
40
jobs_inf_EGamma+Run2018C-UL2018_MiniAODv2-v1+MINIAOD
392
jobs_inf_SUEP-m1000-darkPho+RunIIAutumn18-private+MINIAODSIM
54
jobs_inf_SUEP-m125-generic+RunIIAutumn18-private+MINIAODSIM
4
jobs_inf_SUEP-m1000-darkPhoHad+RunIIAutumn18-private+MINIAODSIM
82
jobs_inf_JetHT+Run2018A-UL2018_MiniAODv2-v1+MINIAOD
4
jobs_inf_SingleMuon+Run2018C-UL2018_MiniAODv2-v2+MINIAOD
12
jobs_inf_JetHT+Run2018B-UL2018_MiniAODv2-v1+MINIAOD
34
jobs_inf_SUEP-m400-generic+RunIIAutumn18-private+MINIAODSIM
54
jobs_inf_SingleMuon+Run2018A-UL2018_MiniAODv2-v3+MINIAOD
468
jobs_inf_EGamma+Run2018B-UL2018_MiniAODv2-v1+MINIAOD
420
jobs_inf_JetHT+Run2018C-UL2018_MiniAODv2-v1+MINIAOD
2
jobs_inf_SingleMuon+Run2018B-UL2018_MiniAODv2-v2+MINIAOD
2
jobs_inf_SUEP-m400-darkPhoHad+RunIIAutumn18-private+MINIAODSIM
48
jobs_inf_SUEP-m1000-generic+RunIIAutumn18-private+MINIAODSIM
20
jobs_inf_SUEP-m750-darkPhoHad+RunIIAutumn18-private+MINIAODSIM
32
jobs_inf_SingleMuon+Run2018D

In [8]:
bad_hosts = []
for dir_name in dirs:
    print(dir_name)
    
    error_files = os.popen('grep '+ err + ' ' + base_dir + dir_name +'/' + job_num + '.*').readlines()
    
    hosts = []
    for entry in error_files:
        if 'os.popen' in entry: continue
        current_file = entry.split('.err')[0]+'.out'
        f2 = open(current_file, "r")
        lines = f2.readlines()
        for line in lines:
            if 'Running on:' not in line: continue
            hostname = line[len("Running on:  "):-1]
            if hostname not in hosts: hosts.append(hostname)
    
    print(hosts, '\n')
    
    for host in hosts:
        if host not in bad_hosts: bad_hosts.append(host)

jobs_2018_data_V6_JetHT+Run2018B-UL2018_MiniAODv2-v1+MINIAOD
['t3btch042.mit.edu'] 



In [42]:
good_hosts = []
for dir_name in dirs:
    print(dir_name)
    
    error_files = os.popen('grep  Tau_jetIdx' + base_dir + dir_name +'/' + job_num + '.*').readlines()

    hosts = []
    for entry in error_files:
        if 'os.popen' in entry: continue
        current_file = entry.split('.err')[0]+'.out'
        f2 = open(current_file, "r")
        lines = f2.readlines()
        for line in lines:
            if 'Running on:' not in line: continue
            hostname = line[len("Running on:  "):-1]
            if hostname not in hosts: hosts.append(hostname)
        
    for host in hosts:
        if host not in good_hosts: good_hosts.append(host)

jobs_2018_MC_V6_QCD_Pt_170to300_TuneCP5_13TeV_pythia8+RunIISummer20UL18MiniAODv2-106X_upgrade2018_realistic_v16_L1v1-v1+MINIAODSIM
[]
jobs_2018_MC_V6_QCD_Pt_2400to3200_TuneCP5_13TeV_pythia8+RunIISummer20UL18MiniAODv2-106X_upgrade2018_realistic_v16_L1v1-v1+MINIAODSIM
[]
jobs_2018_MC_V6_QCD_Pt_30to50_TuneCP5_13TeV_pythia8+RunIISummer20UL18MiniAODv2-106X_upgrade2018_realistic_v16_L1v1-v1+MINIAODSIM
[]
jobs_2018_MC_V6_QCD_Pt_3200toInf_TuneCP5_13TeV_pythia8+RunIISummer20UL18MiniAODv2-pilot_106X_upgrade2018_realistic_v16_L1v1-v1+MINIAODSIM
[]
jobs_2018_MC_V6_QCD_Pt_50to80_TuneCP5_13TeV_pythia8+RunIISummer20UL18MiniAODv2-106X_upgrade2018_realistic_v16_L1v1-v1+MINIAODSIM
[]
jobs_2018_MC_V6_QCD_Pt_1000to1400_TuneCP5_13TeV_pythia8+RunIISummer20UL18MiniAODv2-106X_upgrade2018_realistic_v16_L1v1-v1+MINIAODSIM
[]
jobs_2018_MC_V6_QCD_Pt_120to170_TuneCP5_13TeV_pythia8+RunIISummer20UL18MiniAODv2-106X_upgrade2018_realistic_v16_L1v1-v2+MINIAODSIM
[]
jobs_2018_MC_V6_QCD_Pt_1400to1800_TuneCP5_13TeV_pythia8

In [54]:
print(bad_hosts)

['t2bat0270.cmsaf.mit.edu', 't2bat0256.cmsaf.mit.edu', 't2bat0465.cmsaf.mit.edu', 't2bat0500.cmsaf.mit.edu', 't2bat0461.cmsaf.mit.edu', 't2bat0490.cmsaf.mit.edu', 't2bat0498.cmsaf.mit.edu', 't2bat0497.cmsaf.mit.edu', 't2bat0202.cmsaf.mit.edu', 't2bat0485.cmsaf.mit.edu', 't2bat0079.cmsaf.mit.edu', 't2bat0486.cmsaf.mit.edu', 't2bat0494.cmsaf.mit.edu', 't2bat0240.cmsaf.mit.edu']


In [41]:
print(good_hosts)

[]


In [40]:
# is there any overlap between machines where the jobs
# that ran succesfully and those where they failed?
set(good_hosts).intersection(all_hosts)

set()